In [1]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from vertexai.generative_models import GenerativeModel
from dotenv import load_dotenv

dotenv_path = "../.env"
load_dotenv(dotenv_path)

True

In [2]:
import os

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    verbose=True,
    temperature=0,
    google_api_key=os.getenv("GEMINI_API_KEY"),
)

In [3]:
from crewai_tools import tool
from crewai_tools import SerperDevTool

search_mechanism = SerperDevTool(location="Brazil", hl="pt-br")

@tool("Busca informações na web")
def search_about_movies_in_web(search_query: str):
    """Busca informações na Internet. Exemplo de como usar: search_about_movies_in_web('data estreia <nome-filme> no Brasil em 2024')"""
    search_results = search_mechanism.run(search_query=search_query)
    return search_results

In [4]:
def get_summary_for_movies(movie_names: list[str]):
    search_agent = Agent(
        role="Especialista em Buscar informações sobre estreias de filmes no Brasil nos cinemas",
        goal="Pesquisa sobre filmes que estão para estrear no Brasil nos cinemas",
        backstory="""
              Você é um jornalista de cinema e gosta de filmes mais cults, é muito ligado em premiações.
              Você é especialista em encontrar datas exatas de estreias do filmes no Brasil
        """,
        llm=llm,
        verbose=True,
        tools=[search_about_movies_in_web],
    )

    search_tasks = [
        Task(
            description=f"""Encontrar de todos os filmes especificados especificamente no Brasil no ano de 2024.
                        Devem ser desconsiderados: datas de estreias globais e datas de estreias em festivais, 
                        considere apenas datas que você esteja confiante, se não estiver
                        confiante da data, coloque em release_date que não foi possível encontrar uma data confiável.
                        
                        Filmes Especificados: {movie_name}
                        """,
            agent=search_agent,
            expected_output=f"""
                           Nome do filme, com dia exato estreia para grande público do fime no Brasil em 2024, se não houver
                           preencha release_date dizendo que não foi possível encontrar dado confiável.
                           um comentário com o nível de certeza da data e porquê.
                           Estreias em festivais não devem ser consideradas. 
                           
                           Filmes Especificados: {movie_name}""",
            verbose=True,
        )
        for movie_name in movie_names
    ]

    crew = Crew(
        agents=[search_agent],
        tasks=search_tasks,
        verbose=True,
    )
    content = crew.kickoff()
    return content

In [5]:
result = get_summary_for_movies(["Anora"])

 [DEBUG]: == Working Agent: Especialista em Buscar informações sobre estreias de filmes no Brasil nos cinemas
 [INFO]: == Starting Task: Encontrar de todos os filmes especificados especificamente no Brasil no ano de 2024.
                        Devem ser desconsiderados: datas de estreias globais e datas de estreias em festivais, 
                        considere apenas datas que você esteja confiante, se não estiver
                        confiante da data, coloque em release_date que não foi possível encontrar uma data confiável.
                        
                        Filmes Especificados: Anora
                        


> Entering new CrewAgentExecutor chain...
Thought: Preciso encontrar a data de estreia de "Anora" no Brasil em 2024. Vou procurar informações na web.

Action: Busca informações na web
Action Input: { "search_query": "'data estreia Anora no Brasil em 2024'" }Using Tool: Search the internet
 


Search results: Title: Anora é adiado para 23 de janeiro de 2

In [6]:
result

'Anora, release_date: 23 de janeiro de 2025.\nCerteza: Alta. A data foi confirmada por diversos veículos de comunicação, incluindo Omelete, CNN Brasil, Exame, RedeMLC, Ovício e Rolling Stone. A data original de estreia era 31 de outubro de 2024, mas foi adiada para 23 de janeiro de 2025.'